In [ ]:
## Imports

%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
from PIL import Image

import scipy.misc
from skimage import measure   

import time
import os
import zipfile as zf
import tarfile
import csv
import pickle
import urllib
import pandas as pd


from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

In [2]:
## Dataset Parameters ##
WORKING_DIRECTORY = '/media/dean/datastore/ROAD_COCO/darknet/data'
COCO_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'coco')
IMAGES_DIRECTORY = os.path.join(COCO_DIRECTORY, 'images', 'train2014')
img_prefix = 'COCO_train2014_0000'


UNCHANGED_COCO_ANNOTATIONS_FILE = os.path.join(WORKING_DIRECTORY, 'coco','annotations' , 'instances_train2014.json')
FIXED_COCO_ANNOTATIONS_FILE = os.path.join(WORKING_DIRECTORY, 'coco','annotations' , 'fixed_instances_train2014.json')
ROAD_COCO_ANNOTATIONS_FILE = os.path.join(WORKING_DIRECTORY, 'coco','annotations' , 'roadcoco_instances_train2014.json')


COCO_FIXES_CSV = os.path.join(WORKING_DIRECTORY, 'COCO_Trucks.csv')
HEADER_ROW=['ID', 'FileName', 'ImageUrl', 'KEEP']

COCO_TRUCK_CAR_DELETES_CSV = os.path.join(WORKING_DIRECTORY, 'coco_cars_trucks.csv')

In [3]:
%matplotlib inline
from pycocotools.coco import COCO
import skimage.io as io
import pylab

images_directory = IMAGES_DIRECTORY
road_coco = COCO(UNCHANGED_COCO_ANNOTATIONS_FILE)

categories = road_coco.loadCats(road_coco.getCatIds())
category_names = [category['name'] for category in categories]
print('\nCustom COCO categories: \n{}\n'.format(' '.join(category_names)))

loading annotations into memory...
Done (t=8.30s)
creating index...
index created!

Custom COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush



In [4]:
## Extract Labels ##
image_changes = pd.read_csv(COCO_FIXES_CSV, names=HEADER_ROW, skiprows=1)
image_changes.head()

,ID,FileName,ImageUrl,KEEP
0,155652,COCO_train2014_000000155652.jpg,http://farm3.staticflickr.com/2599/4078412191_...,y
1,253959,COCO_train2014_000000253959.jpg,http://farm4.staticflickr.com/3765/9371427949_...,c
2,253975,COCO_train2014_000000253975.jpg,http://farm1.staticflickr.com/62/211829230_23a...,c
3,188440,COCO_train2014_000000188440.jpg,http://farm2.staticflickr.com/1075/1438769690_...,c
4,417822,COCO_train2014_000000417822.jpg,http://farm9.staticflickr.com/8514/8453715459_...,y


In [ ]:
## DELETE BAD TRUCKS/CHANGE BAD TRUCK LABELS


import json



filename = UNCHANGED_COCO_ANNOTATIONS_FILE
with open(filename, 'r') as f:
    data = json.load(f)
    # Make COCO changes
    for image_change in image_changes.as_matrix():
        
        if image_change[3] == 'n':
            #Get all annotations associated with the image
            anns = road_coco.getAnnIds(imgIds=[image_change[0]])
            
            #Delete all annotations associated with image
            for d in data['annotations']: # Loops through all the nodes in json object
                if d and int(d['id']) in anns:
                    # print('Deleting:', d)
                    data['annotations'].remove(d)
            
        elif image_change[3] == 'c':
            # Change all corresponding labels under image to 'car'
            anns = road_coco.getAnnIds(imgIds=[image_change[0]])
            truck_cat = road_coco.getCatIds(catNms=['truck'])[0]
            car_cat = road_coco.getCatIds(catNms=['car'])[0]
            
            for i, d in enumerate(data['annotations']): # Loops through all the nodes in json object
                for ann in anns:
                    if d and int(d['id']) == int(ann) \
                         and int(d['image_id']) == int(image_change[0]) \
                         and int(d['category_id']) == truck_cat:
                        
                        # print('Changing [', d['category_id'] ,']label to car:', d)
                        data['annotations'][i]['category_id'] = car_cat


os.remove(filename)
with open(FIXED_COCO_ANNOTATIONS_FILE, 'w') as f:
    json.dump(data, f, indent=4)

/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/ipykernel/__main__.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [ ]:
## Extract Labels ##
image_deletions = pd.read_csv(COCO_TRUCK_CAR_DELETES_CSV, names=HEADER_ROW[:-1], skiprows=1)
image_deletions.head()

In [ ]:
## DELETE CARS/TRUCKS IMAGES

import json


filename = FIXED_COCO_ANNOTATIONS_FILE
with open(filename, 'r') as f:
    data = json.load(f)
    # Make COCO changes
    for image_deletion in image_deletions.as_matrix():
        try:
            img_id = int(image_deletion[0])
            #Get all annotations associated with the image
            anns = road_coco.getAnnIds(imgIds=[img_id])
            #Delete all annotations associated with image
            for d in data['annotations']: # Loops through all the nodes in json object
                if d and int(d['id']) in anns:
                    #print('Deleting:', d)
                    data['annotations'].remove(d)
        except ValueError:
            print('Error casting, skipping img_id:', image_deletion[0])
            continue
     
        
        
        


os.remove(filename)
with open(filename, 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
def remove_none(obj):
    if isinstance(obj, (list, tuple, set)):
        return type(obj)(remove_none(x) for x in obj if x is not None)
    elif isinstance(obj, type(dict)):
        return type(obj)((remove_none(k), remove_none(v)) for k, v in obj.items() if k is not None and v is not None)
    else:
        return obj

In [ ]:
import json


filename = FIXED_COCO_ANNOTATIONS_FILE
with open(filename, 'r') as f:
    data = json.load(f)
    #data['annotations'] = remove_none(data['annotations'])
    
    for i, d in enumerate(data['annotations']):
        # Check if width or heigh in bounding box is 0
        if d:
            # xstart, ystart, width, height
            area, bbox = d['area'], d['bbox']
            
            
            if float(bbox[3]) <= 0.0:
                print('Changing BoundingBox/Area Annotations:', bbox,'/',area)
                bbox[3] = 1.1
                data['annotations'][i]['bbox'] = bbox
            if float(bbox[2]) <= 0.0:
                print('Changing BoundingBox/Area Annotations:', bbox,'/',area)
                bbox[2] = 1.1
                data['annotations'][i]['bbox'] = bbox
                
            if float(area) <= 0.0:
                data['annotations'][i]['area'] = float(bbox[2])*float(bbox[3])
                     
        
        
        


os.remove(filename)
with open(filename, 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
import json


filename = FIXED_COCO_ANNOTATIONS_FILE
road_coco = COCO(FIXED_COCO_ANNOTATIONS_FILE) # fixed_instances_annoations file 
truck_cat = road_coco.getCatIds(catNms=['truck'])[0]
car_cat = road_coco.getCatIds(catNms=['car'])[0]
person_cat = road_coco.getCatIds(catNms=['person'])[0]
traffic_light_cat = road_coco.getCatIds(catNms=['traffic light'])[0]
stop_sign_cat = road_coco.getCatIds(catNms=['stop sign'])[0]
parking_meter_cat = road_coco.getCatIds(catNms=['parking meter'])[0]

keep_categories = [truck_cat, person_cat, car_cat, traffic_light_cat, stop_sign_cat, parking_meter_cat]
        
        
with open(filename, 'r') as f:
    data = json.load(f)
    #data['annotations'] = remove_none(data['annotations'])
    
    #Delete all annotations associated with image
    for d in data['annotations']: # Loops through all the nodes in json object
        # Delete Non-ROAD_COCO Categories #
        if d and int(d['category_id']) in keep_categories:
            #print('Skipping:', d['id'], 'with Category ID:', d['category_id'])
            continue
        else:
            #print('Deleting:', d['id'], 'with Category ID:', d['category_id'])
            data['annotations'].remove(d)
                     
        
        
        


os.remove(filename)


# Save as ROADCOCO annotations File
with open(FIXED_COCO_ANNOTATIONS_FILE, 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

In [ ]:
example_coco = COCO(FIXED_COCO_ANNOTATIONS_FILE)

In [ ]:
categories = example_coco.loadCats(example_coco.getCatIds())
# category_names = [category['name'] for category in categories]
# print('Custom COCO categories: \n{}\n'.format(' '.join(category_names)))

# supercategory_names = set([category['supercategory'] for category in categories])
# print('Custom COCO supercategories: \n{}'.format(' '.join(supercategory_names)))
print(categories)

In [ ]:
category_ids = example_coco.getCatIds(catNms=list(category_names))
image_ids = example_coco.getImgIds()

image_data = example_coco.loadImgs(image_ids[np.random.randint(0, len(image_ids))])[0]
print(image_data)

In [ ]:
# load and display instance annotations
image = io.imread(os.path.join(images_directory ,image_data['file_name']))
plt.imshow(image); plt.axis('off')
pylab.rcParams['figure.figsize'] = (68.0, 120.0)
annotation_ids = example_coco.getAnnIds(imgIds=image_data['id'], catIds=category_ids, iscrowd=None)
annotations = example_coco.loadAnns(annotation_ids)
print(annotations)
example_coco.showAnns(annotations)

In [ ]:
ann_ids = example_coco.getAnnIds()
print(len(ann_ids))

In [ ]:
#!java -jar /media/dean/49D9D6EB1BE53FC4/datasets/darknet/data/cocotoyolo.jar "/media/dean/49D9D6EB1BE53FC4/datasets/darknet/data/coco/annotations/fixed_instances_val2014.json" "/media/dean/49D9D6EB1BE53FC4/datasets/darknet/data/coco/images/val2014/" "all" "/media/dean/49D9D6EB1BE53FC4/datasets/darknet/data/coco/labels/fixed_val2014"

In [ ]:
#!/media/dean/49D9D6EB1BE53FC4/datasets/darknet/darknet detector test /media/dean/49D9D6EB1BE53FC4/datasets/darknet/cfg/coco.data /media/dean/49D9D6EB1BE53FC4/datasets/darknet/cfg/yolov3.cfg backup/yolov3.backup -dont_show < /media/dean/49D9D6EB1BE53FC4/datasets/darknet/data/coco/labels/fixed_val2014/image_list.txt > /media/dean/49D9D6EB1BE53FC4/datasets/darknet/results/val_2014_results.json

In [ ]:
#/home/anthony/darknet/darknet detector test /home/anthony/darknet/cfg/coco.data /home/anthony/darknet/cfg/yolov3.cfg backup/yolov3.backup -dont_show < /home/anthony/darknet/data/coco/labels/fixed_val2014/image_list.txt > /home/anthony/darknet/results/val_2014_results.json

In [ ]:
## Imports

%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
from PIL import Image

import scipy.misc
from skimage import measure   

import time
import os
import zipfile as zf
import tarfile
import csv
import pickle
import urllib
import pandas as pd

In [ ]:
## Dataset Parameters ##
images_path = "{}{}".format( IMAGES_DIRECTORY, '*.jpg')
img_prefix = 'COCO_train2014_0000'
    
## VGG Labeler Dataset Extraction Parameters ##
VGG_ANNS_CSV = os.path.join(WORKING_DIRECTORY, 'night_detections.csv')
HEADER_ROW=['filename', 'file_size', 'file_attributes', 'region_count', 'region_id', 'region_shape_attributes', 'region_attributes']

In [ ]:
def maybe_download(source_url, filename):
    if not os.path.exists(IMAGES_DIRECTORY):
        os.mkdir(IMAGES_DIRECTORY)
    
    filepath = os.path.join(IMAGES_DIRECTORY, filename)
    if not os.path.exists(filepath):
        filepath, _ = urllib.request.urlretrieve(source_url, filepath)
        statinfo = os.stat(filepath)
        print('')
        print('Succesfully downloaded:', filepath, '| % d MB.' % int(statinfo.st_size*1e-6))
    return filepath

In [ ]:
## Extract Labels ##
annotations = pd.read_csv(VGG_ANNS_CSV, names=HEADER_ROW, skiprows=1)
annotations.head()

In [ ]:
urlstofilepaths = {}
url_paths = set(annotations['filename'].tolist())

for url in url_paths:
    fname = os.path.split(url)[-1]
    urlstofilepaths[url] = maybe_download(url, os.path.join(IMAGES_DIRECTORY, img_prefix+fname))
print(urlstofilepaths)

In [ ]:
fixed_coco = COCO(FIXED_COCO_ANNOTATIONS_FILE)

categories = fixed_coco.loadCats(fixed_coco.getCatIds())
category_names = [category['name'] for category in categories]
print('Custom COCO categories: \n{}\n'.format(' '.join(category_names)))

In [ ]:
# Script to convert VGG Labeler Annotations to Coco Json Annotations
import xml.etree.cElementTree as ET
import glob
import argparse
import os
import numpy as np
import json
import numpy
import cv2
from collections import OrderedDict
from shapely.geometry import Polygon, MultiPolygon, MultiPoint
import ast

from PIL import Image


# annotations initialized above #

images, anns = [], []
    
img_offset = 3000001

num_imgs = len(url_paths)
ann_index = 3000000
    
for img_id, url in enumerate(sorted(set(url_paths))):
    im = Image.open(urlstofilepaths[url])
    width, height = im.size
    
    if os.path.split(url)[-1].startswith(img_prefix):
        dic = {'file_name': os.path.split(url)[-1], 'id': img_offset+img_id, 'height': height, 'width': width}
    else:
        dic = {'file_name': img_prefix+os.path.split(url)[-1], 'id': img_offset+img_id, 'height': height, 'width': width}
    
    images.append(dic)
    
    for annotation in [x for x in annotations.as_matrix() if x[0].lower() == url.lower()]:

        
        # xy coords: [xstart, ystart, xstop, ystop] -> bbox = [x,y,width,height]
        
        d = ast.literal_eval(annotation[5])
        
        if d:
            if float(d['x']) < 0:
                d['x'] = 0.0
            if float(d['y']) < 0:
                d['y'] = 0.0
            if float(d['height']) <= 0:
                d['height'] = 1.0
                
            if float(d['width']) <= 0:
                d['width'] = 1.0   
            
                
            # Get Points from Bounding Box
            pts = []
            pts.append(((float(d['x']) , float(d['y']))))
            pts.append((float(d['x'])+float(d['width']) , float(d['y'])))
            pts.append((float(d['x'])+float(d['width']) , float(d['y'])+float(d['height'])))
            pts.append((float(d['x']) , float(d['y'])+float(d['height'])))


            # Convert XML Polygon pts to Coco-friendly bounding boxes
            points = MultiPoint(pts)
            xstart,ystart,xstop,ystop = points.bounds

            # Cast to Integers
            xstart,ystart,xstop,ystop = int(xstart),int(ystart),int(xstop),int(ystop)

            binary_mask = np.zeros_like(im)[:,:,0]
            binary_mask[int(xstart):int(xstop), int(ystart):int(ystop)] +=1


            contours = measure.find_contours(binary_mask, 0.5, positive_orientation='low')
            polygons = []
            segmentations = []


            
            for contour in contours:
                poly = Polygon(contour)
                poly = poly.simplify(1.0, preserve_topology=False)

                if poly and poly.exterior:
                    polygons.append(poly)
                    segmentation = np.array(poly.exterior.coords).ravel().tolist()
                    segmentations.append(segmentation)

            # Combine the polygons to calculate the bounding box and area
            multi_poly = MultiPolygon(polygons)
            if multi_poly.bounds:
                x, y, max_x, max_y = multi_poly.bounds
                width = max_x - x
                height = max_y - y
                bbox = (x, y, width, height)
                area = multi_poly.area



                # Get Label
                cls = ast.literal_eval(annotation[6])
                cat = cls['type'].lower().strip()
                
                if cat == 'motorbike':
                    category_id = fixed_coco.getCatIds(catNms=['motorcycle'])
                else:
                    category_id = fixed_coco.getCatIds(catNms=[cat])
                    
                if not category_id: #Hardcode tv for now
                    category_id = fixed_coco.getCatIds(catNms=['tv'])
                
                if type(category_id) == list:
                    category_id = category_id[0]
                
                annotation = {
                    'segmentation': segmentations,
                    'iscrowd': 0,
                    'image_id': img_offset+img_id, # Don't want to conflict with existing dataset
                    'category_id': category_id,
                    'id': ann_index,
                    'bbox': bbox,
                    'area': area
                }
                ann_index+=1
                anns.append(annotation)

In [ ]:
print(anns[:10])

In [ ]:
# Add Existing Coco Dataset
import json


filename = FIXED_COCO_ANNOTATIONS_FILE
        
with open(filename, 'r') as f:
    data = json.load(f)
    print('Before Images Count:', len(images))
    annotated_image_ids = [int(annotation['image_id']) for annotation in data['annotations']]
    # Add Existing Coco Images
    new_images = data['images']
    [images.append(x) for x in new_images if int(x['id']) in annotated_image_ids]
    # Add Existing Coco Annotations
    new_annotations = data['annotations']
    [anns.append(x) for x in new_annotations]
                           
print('Images Count:', len(images))
print('Annotations Count:', len(anns))
    
    

In [ ]:
from datetime import datetime

INFO = {
    "description": "Road Object-Detections Dataset based on MS COCO",
    "url": "https://kache.ai",
    "version": "0.0.1",
    "year": 2018,
    "contributor": "deanwebb",
    "date_created": datetime.utcnow().isoformat(' ')
}

LICENSES = [
    {
        "id": 1,
        "name": "The MIT License (MIT)",
        "url": "https://opensource.org/licenses/MIT",
        "description":  """
                        The MIT License (MIT)
                        Copyright (c) 2017 Matterport, Inc.

                        Permission is hereby granted, free of charge, to any person obtaining a copy
                        of this software and associated documentation files (the "Software"), to deal
                        in the Software without restriction, including without limitation the rights
                        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
                        copies of the Software, and to permit persons to whom the Software is
                        furnished to do so, subject to the following conditions:

                        The above copyright notice and this permission notice shall be included in
                        all copies or substantial portions of the Software.

                        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
                        IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
                        FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
                        AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
                        LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
                        OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
                        THE SOFTWARE.
                        """
    }
]

coco_output = {'info': INFO, 'licenses': LICENSES, 'images':images, 'annotations':anns, 'categories': categories}
with open(ROAD_COCO_ANNOTATIONS_FILE, 'w') as output_json_file:
    json.dump(coco_output, output_json_file)

In [ ]:
testing_coco = COCO(ROAD_COCO_ANNOTATIONS_FILE)

In [ ]:
category_ids = testing_coco.getCatIds(catNms=list(category_names))
image_ids = testing_coco.getImgIds()
image_data = testing_coco.loadImgs(image_ids[np.random.randint(0, len(image_ids))])[0]
print(image_data)

In [ ]:
# load and display instance annotations
image = io.imread(os.path.join(IMAGES_DIRECTORY ,image_data['file_name']))
plt.imshow(image); plt.axis('off')
pylab.rcParams['figure.figsize'] = (98.0, 150.0)
annotation_ids = testing_coco.getAnnIds(imgIds=image_data['id'], catIds=category_ids, iscrowd=None)


annotations = testing_coco.loadAnns(annotation_ids)
print(annotations)
testing_coco.showAnns(annotations)